In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import csv
import warnings
warnings.filterwarnings('ignore')


In [3]:
#Create reference to CSV file
csv_path_people = "../large_data/Traffic_Crashes_-_People.csv"

#Impor the CSV into a pandas DataFrame
df1 = pd.read_csv(csv_path_people) 

In [4]:
#Create reference to CSV file for second file
csv_path_crashes = "../large_data/Traffic_Crashes_-_Crashes.csv"

df2 = pd.read_csv(csv_path_crashes)

In [5]:
#Create reference to CSV file for 3rd file
csv_path_vehicles = "../large_data/Traffic_Crashes_-_Vehicles.csv"

df3 = pd.read_csv(csv_path_vehicles)

In [6]:
df1 = df1[['RD_NO', 'PERSON_ID', 'PERSON_TYPE', 'VEHICLE_ID', 'CRASH_DATE',
       'SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'SEX', 'AGE',
       'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'SAFETY_EQUIPMENT',
       'AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'HOSPITAL',
       'EMS_AGENCY', 'EMS_RUN_NO', 'DRIVER_ACTION', 'DRIVER_VISION',
       'PHYSICAL_CONDITION', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY',
       'PEDPEDAL_LOCATION', 'BAC_RESULT', 'BAC_RESULT VALUE',
       'CELL_PHONE_USE']]
df1.head()

,RD_NO,PERSON_ID,PERSON_TYPE,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,SEX,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,HY368708,O10,DRIVER,10.0,08/04/2015 12:40:00 PM,NaN,CHICAGO,IL,60641,M,...,NaN,FAILED TO YIELD,UNKNOWN,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,HY374018,O100,DRIVER,96.0,07/31/2015 05:50:00 PM,NaN,ELK GROVE,IL,60007,M,...,NaN,FOLLOWED TOO CLOSELY,UNKNOWN,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,HY407431,O1000,DRIVER,954.0,09/02/2015 11:45:00 AM,NaN,CHICAGO,IL,NaN,M,...,NaN,UNKNOWN,UNKNOWN,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,HY484148,O10000,DRIVER,9561.0,10/31/2015 09:30:00 PM,NaN,SKOKIE,IL,60076,M,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,HZ525619,O100001,DRIVER,96762.0,11/15/2016 05:45:00 PM,NaN,NaN,NaN,NaN,X,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [7]:
df3 = df3[['RD_NO', 'CRASH_UNIT_ID', 'CRASH_DATE', 'UNIT_NO', 'UNIT_TYPE',
       'NUM_PASSENGERS', 'VEHICLE_ID', 'CMRC_VEH_I', 'MAKE', 'MODEL',
       'LIC_PLATE_STATE', 'VEHICLE_YEAR', 'VEHICLE_DEFECT', 'VEHICLE_TYPE',
       'VEHICLE_USE', 'TRAVEL_DIRECTION', 'MANEUVER', 'TOWED_I', 'FIRE_I',
       'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I', 'FIRST_CONTACT_POINT']]
df3.head()

,RD_NO,CRASH_UNIT_ID,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,MODEL,...,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,TOWED_I,FIRE_I,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I,FIRST_CONTACT_POINT
0,JB278428,394092,05/24/2018 04:45:00 PM,2,DRIVER,NaN,379191.0,NaN,DODGE,CHARGER,...,NONE,PASSENGER,PERSONAL,W,STRAIGHT AHEAD,NaN,NaN,1.0,NaN,FRONT-LEFT
1,JB278428,394093,05/24/2018 04:45:00 PM,1,PARKED,NaN,379180.0,NaN,PONTIAC,G6,...,NONE,UNKNOWN/NA,UNKNOWN/NA,W,PARKED,NaN,NaN,0.0,NaN,SIDE-LEFT
2,JB281663,396094,05/28/2018 12:30:00 AM,1,DRIVER,NaN,380981.0,NaN,VOLKSWAGEN,JETTA,...,UNKNOWN,PASSENGER,UNKNOWN/NA,W,PASSING/OVERTAKING,NaN,NaN,1.0,NaN,SIDE-RIGHT
3,JB281663,396095,05/28/2018 12:30:00 AM,2,DRIVER,1.0,380986.0,NaN,NISSAN,NISSAN MAXIMA,...,NONE,PASSENGER,RIDESHARE SERVICE,W,STRAIGHT AHEAD,NaN,NaN,2.0,NaN,SIDE-LEFT
4,JB283806,396943,05/29/2018 11:30:00 AM,1,DRIVER,NaN,381829.0,NaN,UNKNOWN,UNKNOWN,...,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,E,STRAIGHT AHEAD,NaN,NaN,1.0,NaN,UNKNOWN


# Load Data Frames into SQlite DB

In [8]:
import sqlalchemy
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import Column, Integer, String, Float


In [9]:
engine = create_engine("sqlite:///chicago_accidents.sqlite", echo=False)
conn = engine.connect()

In [10]:
df1.to_sql('accidents_people', con=engine, index=True, if_exists='replace')

In [11]:
df2.to_sql('accidents_crashes', con=engine, index=True, if_exists='replace')

In [12]:
df3.to_sql('accidents_vehicles', con=engine, index=True, if_exists='replace')